In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [26]:
actuals = pd.read_csv('D:\projects\data\Actual-2019-April.csv')

In [54]:
actuals.head()

,id,timeFrom,energyKwh
0,100,2019-03-31 21:00:00,1790.25
1,100,2019-03-31 21:30:00,1785.00
2,100,2019-03-31 22:00:00,1793.25
3,100,2019-03-31 22:30:00,1778.25
4,100,2019-03-31 23:00:00,1776.00


In [63]:
print(actuals.isna().sum().sum())
print((actuals==0).sum().sum())

# meaning no null values, but alot of 0 values (is it possible?)

0
16611


In [274]:

# lets group by station
pt_by_station = pd.pivot_table(data = actuals, values = 'energyKwh',index = 'timeFrom', columns='id')
pt_by_station['total'] = pt_by_station.sum(axis=1)
pt_by_station *= 2

In [85]:
# the following is a list of station ranked by the number of zero values they have
stations_by_zero_values = (pt_by_station==0).sum(axis=0).sort_values(ascending=False)

# we would like to take out of our calculation any station that has more
# than 500 zero values, as it would inappropriately be marked as 'safe'

uncalculated_meters = stations_by_zero_values[stations_by_zero_values>500].index

In [268]:
# lets get the must meters:
meters_used = actuals['id'].unique()
meters = pd.read_csv(f'..\\data\\OPC-Meters.csv')
must_customers = pd.read_csv(f'..\\data\\must_meters.csv')
must_station = [station for station in meters['name'].values for must in must_customers['Must Customers'].values if must in station]
must_meters = meters[meters['name'].isin(must_station)]['meterId'].values
must_meters_used = [meter for meter in must_meters if meter in meters_used]

In [275]:
pt_by_station_to_rank = pt_by_station.drop(columns=must_meters_used).drop(columns=uncalculated_meters)

In [291]:
# now lets rate the station by the amount they contribute to the total maximum


effect_by_meter = {}
for column in pt_by_station_to_rank.columns:
    max_without = (pt_by_station_to_rank['total']-pt_by_station_to_rank[column]).max() 
    effect_on_max = pt_by_station_to_rank['total'].max() - max_without
    effect_by_meter[column] = effect_on_max

# and rate them by that risk    
meters_by_risk = pd.Series(effect_by_meter).sort_values()[:-1]
meters_by_risk

746                     0.000000
744                     0.000000
782                     0.087500
1024                    0.424000
1062                    0.480000
1052                    0.844000
1058                    0.880000
894                     1.000000
1034                    1.000000
819                     1.325000
908                     2.400000
1048                    2.950000
794                     3.262500
675                     4.000000
1050                    4.750000
1064                    5.840000
795                     6.175000
1047                    7.126000
788                     7.556250
1051                    8.376000
1027                    9.900000
1063                   10.974000
241                    11.000000
1059                   14.138000
1065                   14.400000
585                    14.512500
776                    16.618750
948                    18.475000
1046                   19.574000
1029                   20.034000
          

In [290]:
# doing the same for total consumption might also be right, that way 
# we can measure risk against importance again

pt_by_station_to_rank.sum().sort_values(ascending=False)[1:]

id
CAOL                7.074917e+07
860                 4.118906e+07
311                 1.999379e+07
312                 1.942282e+07
53                  1.710330e+07
242                 1.472970e+07
AP Main             1.168285e+07
52                  8.187424e+06
115                 6.315966e+06
100                 5.316248e+06
50                  4.938784e+06
Clalit Meir MC      4.585378e+06
412                 4.244953e+06
410                 3.788923e+06
117                 3.702151e+06
Clalit Kaplan MC    3.569906e+06
22                  3.513952e+06
25                  3.480788e+06
514                 3.469092e+06
453                 3.157675e+06
106                 3.126366e+06
19                  2.894588e+06
23                  2.832887e+06
663                 2.757328e+06
673                 2.720268e+06
974                 2.707748e+06
107                 2.673568e+06
11                  2.588158e+06
954                 2.587047e+06
1020                2.582919e+06
       

['Gadot Main',
 'Intel F28',
 'Intel IDC1-4',
 'Intel IDC9',
 'Intel IDPj',
 '92',
 'Iscar MainTools',
 '93',
 '91',
 '95',
 'ORL']

In [282]:
rate = 400
meters_included = must_meters_used
for meter in meters_by_risk.index:
    current_max = pt_by_station[meters_included].sum(axis=1).max()
    if current_max > rate*1000:
        break
    meters_included.append(meter)
    

pt_by_station[meters_included].shape

(1441, 545)

In [281]:
meters_by_risk[len(meters_included):]


410                  3015.000000
412                  3023.000000
24                   3094.800000
23                   3122.400000
11                   3130.000000
663                  3221.000000
Clalit Kaplan MC     3223.000000
514                  3390.000000
100                  3555.444680
19                   3692.000000
22                   3889.200000
115                  4318.000000
50                   4334.000000
Clalit Meir MC       4348.000000
52                   6120.944680
AP Main              8406.000000
242                 10416.964463
53                  11648.000000
311                 13960.944680
312                 15008.000000
1056                28205.385180
CAOL                44255.930041
860                 47805.414808
662                 49045.774808
dtype: float64